In [1]:
import sys
import os

# Add the root directory to Python path
os.chdir(os.path.abspath('..'))

# Data preparation

In [2]:
from src.data_loader.taiwanchat_loader import TaiwanChatDataLoader
from src.data_processor.message_handler import (
    format_fine_tune_dataset_as_messages,
    check_openai_format_errors,
    num_tokens_from_messages,
    format_fine_tune_dataset_as_openai_input,
    format_fine_tune_dataset_as_openai_input_with_threshold
)

c:\Users\l501l\Desktop\TaiwanGPT\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
twchat_loader = TaiwanChatDataLoader("yentinglin/TaiwanChat", cache_dir='./data/fine_tuning')
twchat_dataset = twchat_loader.load_dataset()
filtered_dataset = twchat_dataset.filter(lambda example: example['dataset_name'] == 'sharegpt')

Dataset already exists at ./data/fine_tuning\yentinglin/TaiwanChat. Loading from disk...


In [4]:
openai_input = format_fine_tune_dataset_as_openai_input_with_threshold(filtered_dataset, 2)
check_openai_format_errors(openai_input)

No errors found


In [5]:
token_list = [num_tokens_from_messages(message['messages']) for message in openai_input]
sum(token_list)

1999693

In [6]:
import json
from datetime import datetime
from typing import List, Dict

def export_to_json(data: List[Dict], base_filename: str, output_dir: str = "."):
    """
    Exports data to a JSON file with a timestamped suffix.
    
    Args:
        data (List[Dict]): The data to be exported.
        base_filename (str): The base name for the JSON file.
        output_dir (str): The directory where the JSON file will be saved.
    """
    # Generate the current timestamp in the format YYYY-MM-DD_HH-MM-SS
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    # Create the full filename with the base name and timestamp
    filename = f"{base_filename}_{timestamp}.json"
    filepath = f"{output_dir}/{filename}"
    
    # Write the data to a JSON file
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    print(f"Data exported to {filepath}")


In [7]:
export_to_json(openai_input, 'Taiwan_Chat_sharegpt_2M', './data/fine_tuning/openai')

Data exported to ./data/fine_tuning/openai/Taiwan_Chat_sharegpt_2M_2024-09-22_20-50-42.json


# Upload a training file

In [ ]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="file-abc123",
  model="gpt-4o-mini"
)

#　Create a fine-tuned model

# Create a fine-tuned model